In [2]:
!pip install --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib

import re
from dataclasses import dataclass
from typing import List, Tuple

from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# === 1. AUTHENTICATION & SERVICE CREATION ====================================

def get_docs_service():
    """
    Authenticate the user in Colab and return an authorized Google Docs service.
    """
    try:
        auth.authenticate_user()
        credentials, _ = google.auth.default()
        service = build("docs", "v1", credentials=credentials)
        return service
    except Exception as e:
        raise RuntimeError(f"Failed to authenticate with Google API: {e}")


# === 2. MARKDOWN SOURCE ======================================================

MEETING_MARKDOWN = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.41.1 which is incompatible.
google-cloud-core 1.7.1 requires google-api-core<2.0.0dev,>=1.21.0, but you have google-api-core 2.28.1 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.41.1 which is incompatible.


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
def build_requests_from_markdown(markdown: str, start_index: int = 1):
    """
    Single-pass line-by-line builder.

    For each line:
      - Decide type (heading, bullet, checkbox, footer, paragraph)
      - Insert text
      - Immediately add style/bullet requests on that [start, end] range

    Returns:
      - requests: list of batchUpdate requests
    """
    requests = []
    cursor = start_index
    in_footer = False
    mention_pattern = re.compile(r"@\w+")

    lines = markdown.splitlines()

    for raw_line in lines:
        line = raw_line.rstrip("\n")

        # --- Footer separator ---
        if line.strip() == "---":
            in_footer = True
            continue

        # --- Blank line: just insert a newline paragraph, no styling ---
        if not line.strip():
            text = "\n"
            start = cursor
            end = cursor + len(text)

            requests.append(
                {
                    "insertText": {
                        "location": {"index": cursor},
                        "text": text,
                    }
                }
            )
            cursor = end
            continue

        # --- If we're past '---', treat as footer line ---
        if in_footer:
            text = line.strip() + "\n"
            start = cursor
            end = cursor + len(text)

            requests.append(
                {
                    "insertText": {
                        "location": {"index": cursor},
                        "text": text,
                    }
                }
            )

            # Footer style: italic + gray
            requests.append(
                {
                    "updateTextStyle": {
                        "range": {"startIndex": start, "endIndex": end},
                        "textStyle": {
                            "italic": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {
                                        "red": 0.4,
                                        "green": 0.4,
                                        "blue": 0.4,
                                    }
                                }
                            },
                        },
                        "fields": "italic,foregroundColor",
                    }
                }
            )

            cursor = end
            continue

        if line.startswith("# "):
            main_text = line[2:].strip()
            if " - " in main_text:
                title_part, date_part = main_text.split(" - ", 1)

                # Insert title as Heading 1
                title_text = title_part.strip() + "\n"
                t_start = cursor
                t_end = cursor + len(title_text)
                requests.append(
                    {
                        "insertText": {
                            "location": {"index": cursor},
                            "text": title_text,
                        }
                    }
                )
                requests.append(
                    {
                        "updateParagraphStyle": {
                            "range": {"startIndex": t_start, "endIndex": t_end},
                            "paragraphStyle": {"namedStyleType": "HEADING_1"},
                            "fields": "namedStyleType",
                        }
                    }
                )
                cursor = t_end

                # Insert date as normal paragraph
                date_text = date_part.strip() + "\n"
                d_start = cursor
                d_end = cursor + len(date_text)
                requests.append(
                    {
                        "insertText": {
                            "location": {"index": cursor},
                            "text": date_text,
                        }
                    }
                )
                cursor = d_end

            else:
                # Simple H1
                text = main_text + "\n"
                start = cursor
                end = cursor + len(text)
                requests.append(
                    {
                        "insertText": {
                            "location": {"index": cursor},
                            "text": text,
                        }
                    }
                )
                requests.append(
                    {
                        "updateParagraphStyle": {
                            "range": {"startIndex": start, "endIndex": end},
                            "paragraphStyle": {"namedStyleType": "HEADING_1"},
                            "fields": "namedStyleType",
                        }
                    }
                )
                cursor = end

            continue

        if line.startswith("## "):
            text = line[3:].strip() + "\n"
            start = cursor
            end = cursor + len(text)

            requests.append(
                {
                    "insertText": {
                        "location": {"index": cursor},
                        "text": text,
                    }
                }
            )
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start, "endIndex": end},
                        "paragraphStyle": {"namedStyleType": "HEADING_2"},
                        "fields": "namedStyleType",
                    }
                }
            )
            cursor = end
            continue

        if line.startswith("### "):
            text = line[4:].strip() + "\n"
            start = cursor
            end = cursor + len(text)

            requests.append(
                {
                    "insertText": {
                        "location": {"index": cursor},
                        "text": text,
                    }
                }
            )
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start, "endIndex": end},
                        "paragraphStyle": {"namedStyleType": "HEADING_3"},
                        "fields": "namedStyleType",
                    }
                }
            )
            cursor = end
            continue

        # Checkboxes"
        m_checkbox = re.match(r"^(\s*)-\s+\[\s\]\s+(.*\S.*)$", line)
        if m_checkbox:
            indent_spaces = len(m_checkbox.group(1))
            level = indent_spaces // 2
            content = m_checkbox.group(2).strip()

            text = content + "\n"
            start = cursor
            end = cursor + len(text)

            # Insert text
            requests.append(
                {
                    "insertText": {
                        "location": {"index": cursor},
                        "text": text,
                    }
                }
            )

            requests.append(
                {
                    "createParagraphBullets": {
                        "range": {"startIndex": start, "endIndex": end},
                        "bulletPreset": "BULLET_CHECKBOX",
                    }
                }
            )

            # Visual nesting: indentStart + hanging indent
            indent_start_pt = 18 + level * 18
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start, "endIndex": end},
                        "paragraphStyle": {
                            "indentStart": {
                                "magnitude": indent_start_pt,
                                "unit": "PT",
                            },
                            "indentFirstLine": {
                                "magnitude": -18,
                                "unit": "PT",
                            },
                        },
                        "fields": "indentStart,indentFirstLine",
                    }
                }
            )

            # Style @mentions inside checkbox line
            for match in mention_pattern.finditer(content):
                mention_start = start + match.start()
                mention_end = start + match.end()
                requests.append(
                    {
                        "updateTextStyle": {
                            "range": {
                                "startIndex": mention_start,
                                "endIndex": mention_end,
                            },
                            "textStyle": {
                                "bold": True,
                                "foregroundColor": {
                                    "color": {
                                        "rgbColor": {
                                            "red": 0.0,
                                            "green": 0.4,
                                            "blue": 0.8,
                                        }
                                    }
                                },
                            },
                            "fields": "bold,foregroundColor",
                        }
                    }
                )

            cursor = end
            continue

        # --- Regular bullets: "-" or "*" ---
        m_bullet = re.match(r"^(\s*)[-*]\s+(.*\S.*)$", line)
        if m_bullet:
            indent_spaces = len(m_bullet.group(1))
            level = indent_spaces // 2
            content = m_bullet.group(2).strip()
            if level == 0:
                content = "\t"*level + content
            text = content + "\n"
            start = cursor + 2*level
            end = cursor + len(text)

            # Insert text
            requests.append(
                {
                    "insertText": {
                        "location": {"index": cursor},
                        "text": text,
                    }
                }
            )

            if level == 0:
                bullet_preset = "BULLET_DISC_CIRCLE_SQUARE"
            elif level == 1:
                bullet_preset = "BULLET_DIAMOND_CIRCLE_SQUARE"
            else:
                bullet_preset = "BULLET_ARROW_DIAMOND_DISC"

            # Make it a normal bullet
            requests.append(
                {
                    "createParagraphBullets": {
                        "range": {"startIndex": start, "endIndex": end},
                        "bulletPreset": bullet_preset,
                    }
                }
            )

            # Visual nesting
            indent_start_pt = 18 + level * 18
            requests.append(
                {
                    "updateParagraphStyle": {
                        "range": {"startIndex": start, "endIndex": end},
                        "paragraphStyle": {
                            "indentStart": {
                                "magnitude": indent_start_pt,
                                "unit": "PT",
                            },
                            "indentFirstLine": {
                                "magnitude": -18,
                                "unit": "PT",
                            },
                        },
                        "fields": "indentStart,indentFirstLine",
                    }
                }
            )

            cursor = end
            continue

        # --- Normal paragraph line (non-footer) ---
        text = line.strip() + "\n"
        start = cursor
        end = cursor + len(text)

        requests.append(
            {
                "insertText": {
                    "location": {"index": cursor},
                    "text": text,
                }
            }
        )

        # Style @mentions in normal paragraphs too, if any
        for match in mention_pattern.finditer(line.strip()):
            mention_start = start + match.start()
            mention_end = start + match.end()
            requests.append(
                {
                    "updateTextStyle": {
                        "range": {
                            "startIndex": mention_start,
                            "endIndex": mention_end,
                        },
                        "textStyle": {
                            "bold": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {
                                        "red": 0.0,
                                        "green": 0.4,
                                        "blue": 0.8,
                                    }
                                }
                            },
                        },
                        "fields": "bold,foregroundColor",
                    }
                }
            )

        cursor = end

    return requests

def main():
    """
    Main function:
      1. Authenticate and create Docs service
      2. Create a new Google Doc
      3. Build all insert + format requests line-by-line
      4. Apply them in a single batchUpdate
    """
    docs_service = get_docs_service()

    doc_title = "Product Team Sync - May 15, 2023"
    document_id = create_document(docs_service, doc_title)

    requests = build_requests_from_markdown(MEETING_MARKDOWN, start_index=1)
    apply_requests(docs_service, document_id, requests)

    print(f"All done! Open your document here: https://docs.google.com/document/d/{document_id}/edit")

if __name__ == "__main__":
    main()

NameError: name 'get_docs_service' is not defined